In [ ]:
# load packages
!pip install mapie
!pip install shap
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_percentage_error as mape, mean_squared_error as mse
from pyarrow import feather as pq
import geopandas as gpd
import folium
from folium import Marker
from shapely import geometry
from tqdm import tqdm
pd.set_option('display.max_columns', None)
from ipywidgets import interact
import scipy
from tqdm import tqdm
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
import mapie
from mapie import regression
from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.regression import MapieRegressor
from mapie.quantile_regression import MapieQuantileRegressor
from sklearn.ensemble import GradientBoostingRegressor
import shap
# Feature Importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
# scaler for Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# mount drive
from google.colab import drive
drive.mount("/content/gdrive")
# load data for monthly performance
data = pd.read_csv("/content/gdrive/MyDrive/Aurora_Thesis/data_converted.csv")

In [ ]:
# load data
data.time = pd.to_datetime(data.time)
data = data.reset_index()
# need to manually add 2 station Id
data.loc[data.station== "Bologna (BO)", "station_id"] = "ID1999"
data.loc[data.station== "San Pietro Capofiume (SPC)", "station_id"] = "ID1998"
# switch London data
data["OAtot_2"] = data.HOA_PMF + data.BBOA_PMF + data.OOAtot_PMF
data.loc[data.station == "London","OAtot_PMF"] = data.loc[data.station == "London","OAtot_2"]
# Remove Zurich 2017
data = data.loc[(data.station != "Zurich") | (data.year != 2017),:]

In [ ]:
# CLEAN DATA
# remove OA with less than 0.1
data= data.loc[data.OAtot_PMF >= 0.1, :]
# and stations with less than 30 obs.
select = (data.groupby("station_id")["OAtot_PMF"].size() > 30).reset_index()
data = data.set_index("station_id")
data = data.join(select.set_index("station_id"), rsuffix = "keep")
data = data.loc[ data.OAtot_PMFkeep == True, :]
data = data.reset_index()

In [ ]:
# add day of week
data["day_week"] = data.time.dt.day_of_week

In [ ]:
# feature engineering
data["rc_1_1000-rc_1_100"] = data["road_class_1_1000"] - data["road_class_1_100"]
data["rc_2_1000-rc_2_100"] = data["road_class_2_1000"] - data["road_class_2_100"]
data["rc_3_1000-rc_3_100"] = data["road_class_3_1000"] - data["road_class_3_100"]
# CAMX proportions of components of OA
data["p_HOA"] = data["HOA_CAMX"] / data["OAtot_CAMX"]
data["p_BBOA"] = data["BBOA_CAMX"] / data["OAtot_CAMX"]
data["p_OOAtot"] = data["OOAtot_CAMX"] / data["OAtot_CAMX"]

# need to decorralate some land-use variables
data["diff_agriculture"] = data["agriculture1000"] - data["agriculture500"]
data["diff_airports"] = data["airports1000"] - data["airports500"]
data["diff_barren"] = data["barren1000"] - data["barren500"]
data["diff_industrial"] = data["industrial1000"] - data["industrial500"]
data["diff_industrial_transport"]= data["industrial_transport1000"] - data["industrial_transport500"]
data["diff_natural_green"] =  data["natural_green1000"] - data["natural_green500"]
data["diff_ports"] = data["ports1000"] - data["ports500"]
data["diff_roads_rails"] = data["roads_rails1000"] - data["roads_rails500"]
data["diff_snow_ice"] = data["snow_ice1000"] - data["snow_ice500"]
data["diff_transport"] = data["transport1000"] - data["transport500"]
data["diff_urban_fabric"] = data["urban_fabric1000"] - data["urban_fabric500"]
data["diff_urban_green"] = data["urban_green1000"] - data["urban_green500"]
data["diff_water"] = data["water1000"] - data["water500"]
data["diff_wetlands"] = data["wetlands1000"] - data["wetlands500"]
# also for Population and IMD
data["diff_population"] = data["population_1000"] - data["population_500"]
data["diff_imd"] = data["imd1000"] - data["imd500"]

In [ ]:
def my_mape(Y_true, Y_pred):
    loss = (np.abs( (Y_true - Y_pred)/(Y_true + 1))).mean()
    return loss

# Conformal Prediction at unseen locations

In [ ]:
# CP for unseen locations
# Predict at unseen stations
np.random.seed(1)
n_sp = data.station_id.nunique() # try n_sp equal the number of unique stations
# for RF try n_split = number of stations
cv = GroupKFold(n_splits = n_sp)
res_rf = {}
res_boost = {}
res_ridge = {}
y_pis = {}
y_pis_pred = {}
y_pis_cqr = {}
y_cqr_pred = {}
y_qr_lo = {}
y_qr_up= {}
covars = ["HOA_CAMX", "BBOA_CAMX", "OOAtot_CAMX", "year", "month","day_week",
"temp_CAMX", "rh_CAMX", "press_CAMX", "ws_CAMX", "wd_CAMX", "pblh_CAMX", "wind_x_CAMX", "wind_y_CAMX",
    "agriculture500","airports500", "barren500", "industrial500", "industrial_transport500", "natural_green500", "ports500", "roads_rails500", "snow_ice500",
    "transport500", "urban_fabric500","urban_green500", "water500", "wetlands500","imd500", "population_500","elevation","Lat","Lon", "area_grid",
    "distance_border", "distance_mt"]
    # "diff_agriculture", "diff_airports", "diff_barren", "diff_industrial", "diff_industrial_transport",
    # "diff_natural_green", "diff_ports", "diff_roads_rails", "diff_snow_ice", "diff_transport", "diff_urban_fabric", "diff_urban_green", "diff_water", "diff_wetlands","diff_population", "diff_imd"
X = data.loc[:, covars]
X = X.fillna(0)
Y = data.loc[:,"OAtot_PMF"]
station_id = data.loc[:, "station_id"]
alpha = 0.1
# Grouped Cross-Validation
for idx, (train_idx, test_idx) in tqdm(enumerate(cv.split(X, Y, station_id))):
    # get train ad test folds
    X_train = X.iloc[train_idx, :]
    Y_train = Y.iloc[train_idx]
    X_test = X.iloc[test_idx, :]
    Y_test = Y.iloc[test_idx]
    # split training in proper train and calibration set (divide stations using groupshuffle function)
    train_inds, test_inds =next(GroupShuffleSplit(random_state = 12).split(X_train, Y_train, data.station_id.iloc[train_idx]))
    X_train, X_cal, Y_train, Y_cal = X_train.iloc[train_inds,:], X_train.iloc[test_inds,:], Y_train.iloc[train_inds], Y_train.iloc[test_inds]

    # Split CQR
    bb = GradientBoostingRegressor(loss = "quantile", alpha = 0.5, random_state=1)
    bb.fit(X_train, Y_train)
    alpha = 0.1
    mapie = MapieRegressor(bb, cv = "prefit", method = "base")
    mapie.fit(X_cal, Y_cal)

    # QUANTILE REGRESSION
    lo_boost = GradientBoostingRegressor(loss = "quantile", alpha = alpha/2, random_state=1)
    up_boost = GradientBoostingRegressor(loss = "quantile", alpha = 1-alpha/2, random_state=1)
    lo_boost.fit(X_train, Y_train)
    up_boost.fit(X_train, Y_train)

    # CQR
    median_boost = GradientBoostingRegressor(loss = "quantile", alpha = 0.5, random_state=1)
    mapie_cqr = MapieQuantileRegressor(median_boost, cv = "split", method = "quantile", alpha = alpha)
    mapie_cqr.fit(X_train, Y_train,
                    X_calib = X_cal, y_calib = Y_cal)


    # Predict at unseen station
    test_stations = station_id.iloc[test_idx].unique()
    for test_id in test_stations:
        res_rf[test_id] = bb.predict(X.loc[data.station_id == test_id,:])
        y_pis_pred[test_id], y_pis[test_id] =  mapie.predict(X.loc[data.station_id == test_id,:], alpha = alpha)
        y_cqr_pred[test_id], y_pis_cqr[test_id] =  mapie_cqr.predict(X.loc[data.station_id == test_id,:], alpha = alpha)
        y_qr_lo[test_id] = lo_boost.predict(X_test)
        y_qr_up[test_id] = up_boost.predict(X_test)


102it [2:29:02, 87.67s/it]


In [ ]:
def plot_comparison(station_name):
    #sns.set_style("whitegrid")
    station_id = data.loc[data.station == station_name, "station_id"].iloc[0]
    y_obs = data.loc[data.station_id == station_id, "OAtot_PMF"].reset_index(drop=True)
    y_camx = (data.loc[data.station_id == station_id,["HOA_CAMX","BBOA_CAMX","OOAtot_CAMX"]].apply(np.sum, axis = 1)).reset_index(drop=True)
    fig, ax = plt.subplots(1,2, figsize = (20,9), sharey = True)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor = "black")
    y_loc = np.max( (y_obs.max(), y_camx.max()) ) * 0.9
    # NAIVE
    ax[0].plot(res_rf[station_id], label = "Prediction", color = "green")
    ax[0].plot(y_obs, label ="Observed OA", color = "blue")
    ax[0].plot(y_camx, label ="CAMx", color = "orange")
    ax[0].fill_between(y_obs.index, y_pis[station_id][:,0,0], y_pis[station_id][:,1,0], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[0].set_title("Standard CP", fontsize = 20)
    ax[0].set_ylabel("OA", fontsize = 20)
    ax[0].set_yticklabels(ax[0].get_yticks(),fontsize = 16)
    #ax[0].set_xticklabels(ax[0].get_xticks(),fontsize = 16)
    ax[0].set_xlabel("Time",fontsize = 16)
    naive_coverage = np.round(regression_coverage_score(y_obs, y_pis[station_id][:,0,0], y_pis[station_id][:,1,0]),2)
    naive_width = (y_pis[station_id][:,1,0] - y_pis[station_id][:,0,0]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (naive_coverage, ),
        r"$Width=%.2f$" % (naive_width, )))
    ax[0].text(1,y_loc, textstr, bbox = props,fontsize = 20)
    # CQR
    ax[1].plot(res_rf[station_id], label = "Prediction", color = "green")
    ax[1].plot(y_obs, label ="Observed OA", color = "blue")
    ax[1].plot(y_camx, label ="CAMx", color = "orange")
    ax[1].fill_between(y_obs.index, y_pis_cqr[station_id][:,0,0], y_pis_cqr[station_id][:,1,0], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[1].set_title("CQR", fontsize = 20)
    cqr_coverage = np.round(regression_coverage_score(y_obs, y_pis_cqr[station_id][:,0,0], y_pis_cqr[station_id][:,1,0]),2)
    cqr_width = (y_pis_cqr[station_id][:,1,0] - y_pis_cqr[station_id][:,0,0]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (cqr_coverage, ),
        r"$Width=%.2f$" % (cqr_width, )))
    ax[1].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    ax[1].set_yticklabels(ax[1].get_yticks(),fontsize = 16)
    #ax[1].set_xticklabels(ax[1].get_xticks(),fontsize = 16)$
    ax[1].set_xlabel("Time",fontsize = 16)
    plt.legend(loc='upper center', bbox_to_anchor=(-0.1, -0.15),
                    fancybox=True, shadow=True, ncol=5, fontsize = 20)


    fig.suptitle(station_id + ": " + station_name, fontsize = 24)
    for ax in ax.flat:
        ax.label_outer()

In [ ]:
interact(plot_comparison, station_name = data.station.unique())

In [ ]:
# get results of average coverage, average normalized width and then station spread in coverage using varius metricls like std, quantiles

# dict to save results
w_naive = {}
cov_naive = {}
w_cqr = {}
cov_cqr = {}
w_qr = {}
cov_qr = {}
for stat in data.station_id.unique():
  y_test = data.loc[data.station_id == stat, "OAtot_PMF"]
  w_naive[stat] = regression_mean_width_score(y_pis[stat][:,0,0], y_pis[stat][:,1,0])/(y_test.max()-y_test.min())
  cov_naive[stat] = regression_coverage_score(y_test, y_pis[stat][:,0,0], y_pis[stat][:,1,0] )
  w_cqr[stat] = regression_mean_width_score(y_pis_cqr[stat][:,0,0], y_pis_cqr[stat][:,1,0])/(y_test.max()-y_test.min())
  cov_cqr[stat] = regression_coverage_score(y_test, y_pis_cqr[stat][:,0,0], y_pis_cqr[stat][:,1,0] )
  w_qr[stat] = regression_mean_width_score(y_qr_lo[stat], y_qr_up[stat])/(y_test.max()-y_test.min())
  cov_qr[stat] = regression_coverage_score(y_test, y_qr_lo[stat], y_qr_up[stat] )

In [ ]:
# unweighted mean
print("avg. width naive: ", np.mean(list(w_naive.values())))
print("avg. cov naive: ", np.mean(list(cov_naive.values())))
print("avg. width cqr: ", np.mean(list(w_cqr.values())))
print("avg. cov cqr: ", np.mean(list(cov_cqr.values())))
print("avg. width qr: ", np.mean(list(w_qr.values())))
print("avg. cov qr: ", np.mean(list(cov_qr.values())))

In [ ]:
# unweighted median
print("avg. width naive: ", np.median(list(w_naive.values())))
print("avg. cov naive: ", np.median(list(cov_naive.values())))
print("avg. width cqr: ", np.median(list(w_cqr.values())))
print("avg. cov cqr: ", np.median(list(cov_cqr.values())))
print("avg. width qr: ", np.median(list(w_qr.values())))
print("avg. cov qr: ", np.median(list(cov_qr.values())))

In [ ]:
# weighted mean with loop (RUN THIS!)
w_naivef = 0
cov_naivef = 0
w_cqrf = 0
cov_cqrf = 0
w_qrf = 0
cov_qrf = 0
for stat in data.station_id.unique():
  n = len(data.loc[data.station_id == stat, "OAtot_PMF"])
  w_naivef += w_naive[stat]*n
  cov_naivef += cov_naive[stat]*n
  w_cqrf += w_cqr[stat]*n
  cov_cqrf += cov_cqr[stat]*n
  w_qrf += w_qr[stat]*n
  cov_qrf += cov_qr[stat]*n

In [ ]:
# weighted mean
N = len(data.OAtot_PMF)
print("avg. width naive: ", w_naivef/N)
print("avg. cov naive: ", cov_naivef/N)
print("avg. width cqr: ", w_cqrf/N)
print("avg. cov cqr: ", cov_cqrf/N)
print("avg. width qr: ", w_qrf/N)
print("avg. cov qr: ", cov_qrf/N)

In [ ]:
# unweighted stadard deviation
print("std. cov naive: ", np.std(list(cov_naive.values())))
print("std. cov cqr: ", np.std(list(cov_cqr.values())))
print("std. cov qr: ", np.std(list(cov_qr.values())))
# unweighted stadard deviation
print("std. w naive: ", np.std(list(w_naive.values())))
print("std. w cqr: ", np.std(list(w_cqr.values())))
print("std. w qr: ", np.std(list(w_qr.values())))

In [ ]:
from matplotlib.offsetbox import AnnotationBbox, TextArea
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import randint, uniform
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from mapie.metrics import (regression_coverage_score,
                           regression_mean_width_score)
from mapie.subsample import Subsample
random_state = 23
rng = np.random.default_rng(random_state)
round_to = 3
warnings.filterwarnings("ignore")

In [ ]:
# plot all methods x vs y randomly getting stations
def sort_y_values(y_test, y_pred, y_pis):
    """
    Sorting the dataset in order to make plots using the fill_between function.
    """
    indices = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[indices]
    y_pred_sorted = y_pred[indices]
    y_lower_bound = y_pis[:, 0, 0][indices]
    y_upper_bound = y_pis[:, 1, 0][indices]
    return y_test_sorted, y_pred_sorted, y_lower_bound, y_upper_bound

def sort_y_values2(y_test, y_pred, y_lower, y_upper):
    """
    Sorting the dataset in order to make plots using the fill_between function.
    """
    indices = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[indices]
    y_pred_sorted = y_pred[indices]
    y_lower_bound = y_lower[indices]
    y_upper_bound = y_upper[indices]
    return y_test_sorted, y_pred_sorted, y_lower_bound, y_upper_bound


def plot_prediction_intervals(
    title,
    axs,
    y_test_sorted,
    y_pred_sorted,
    lower_bound,
    upper_bound,
    coverage,
    width,
    num_plots_idx
):
    """
    Plot of the prediction intervals for each different conformal
    method.
    """

    axs.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    axs.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))

    lower_bound_ = np.take(lower_bound, num_plots_idx)
    y_pred_sorted_ = np.take(y_pred_sorted, num_plots_idx)
    y_test_sorted_ = np.take(y_test_sorted, num_plots_idx)

    error = np.abs(y_pred_sorted_-lower_bound_)

    warning1 = y_test_sorted_ > y_pred_sorted_+error
    warning2 = y_test_sorted_ < y_pred_sorted_-error
    warnings = warning1 + warning2
    axs.errorbar(
        y_test_sorted_[~warnings],
        y_pred_sorted_[~warnings],
        yerr=error[~warnings],
        capsize=5, marker="o", elinewidth=2, linewidth=0, color = "blue",
        label="Inside prediction interval"
        )
    axs.errorbar(
        y_test_sorted_[warnings],
        y_pred_sorted_[warnings],
        yerr=error[warnings],
        capsize=5, marker="o", elinewidth=2, linewidth=0, color="red",
        label="Outside prediction interval"
        )
    axs.scatter(
        y_test_sorted_[warnings],
        y_test_sorted_[warnings],
        marker="*", color="green",
        label="True value"
    )
    axs.set_xlabel("Observed OA")
    axs.set_ylabel("Predicted OA")
    #ab = AnnotationBbox(TextArea(
           # f"Coverage: {np.round(coverage, round_to)}\n"
          #  + f"Interval width: {np.round(width, round_to)}"),xy=(np.min(y_test_sorted_)*3, np.max(y_pred_sorted_+error)*0.95),)

    lims = [
        np.min([axs.get_xlim(), axs.get_ylim()]),  # min of both axes
        np.max([axs.get_xlim(), axs.get_ylim()]),  # max of both axes
    ]
    pt = (0, 0)
    axs.axline(pt, slope=1, color='black', label = "x=y")
    #axs.plot(0,1, '--', alpha=0.75, color="black", label="x=y")
    #axs.add_artist(ab)
    axs.set_title(title, fontweight='bold', fontsize = 20)
    axs.set_ylabel(axs.get_ylabel(), fontsize = 18)
    axs.set_xlabel(axs.get_xlabel(), fontsize = 18)
    # change the fontsize
    axs.tick_params(axis='x', labelsize=16)
    axs.tick_params(axis='y', labelsize=16)
    #fig.supxlabel('common_x')
    #fig.supylabel('common_y')

In [ ]:
def plot_stat(station_name):
  stat_id = data.loc[data.station == station_name, "station_id"].iloc[0]
  y_test = data.loc[data.station_id == stat_id, "OAtot_PMF"]
  y_test_sorted = {}
  y_pred_sorted = {}
  y_lower_sorted = {}
  y_upper_sorted = {}
  y_test_sorted["Standard CP"] , y_pred_sorted["Standard CP"], y_lower_sorted["Standard CP"], y_upper_sorted["Standard CP"] = sort_y_values(y_test, y_cqr_pred[stat_id], y_pis[stat_id])
  y_test_sorted["CQR"] , y_pred_sorted["CQR"], y_lower_sorted["CQR"], y_upper_sorted["CQR"]= sort_y_values(y_test, y_cqr_pred[stat_id], y_pis_cqr[stat_id])


  fig, ax = plt.subplots(1,2, figsize = (16,9), sharey = True)
  sns.set_style("whitegrid")
  axs = [ ax[0], ax[1] ]
  strategy = ["Standard CP", "CQR"]
  perc_obs_plot = 1
  num_plots = rng.choice(len(y_test), int(perc_obs_plot*len(y_test)), replace=False)
  for axis, strat in zip(axs, strategy):
    plot_prediction_intervals(
                    strat,
                    axis,
                    y_test_sorted[strat],
                    y_pred_sorted[strat],
                    y_lower_sorted[strat],
                    y_upper_sorted[strat],
                    10,
                    2,
                    num_plots
                    )

  lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
  lines, labels = [sum(_, []) for _ in zip(*lines_labels)]
  plt.legend(
                lines[:4], labels[:4],
                loc='upper center',
                bbox_to_anchor=(0, -0.225),
                fancybox=True,
                shadow=True,
                ncol=2, fontsize = 20
            )
  for ax in ax.flat:
        ax.label_outer()
  plt.suptitle(station_name, fontsize = 30)
  plt.show()

In [ ]:
interact(plot_stat, station_name = data.station.unique())

In [ ]:
# multiple stations plot
fig, ax = plt.subplots(1,2, figsize = (20,9), sharey = True)
sns.set_style("whitegrid")
axs = [ax[0], ax[1]]
strategy = ["Standard CP", "CQR"]
perc_obs_plot = 0.01

for idx, stat_id in enumerate(data.station_id.unique()):
    y_test = data.loc[data.station_id == stat_id, "OAtot_PMF"]
    num_plots = rng.choice(len(y_test), int(perc_obs_plot*len(y_test)), replace=False)
    y_test_sorted = {}
    y_pred_sorted = {}
    y_lower_sorted = {}
    y_upper_sorted = {}
    y_test_sorted["Standard CP"] , y_pred_sorted["Standard CP"], y_lower_sorted["Standard CP"], y_upper_sorted["Standard CP"] = sort_y_values(y_test, y_pis_pred[stat_id], y_pis[stat_id])
    y_test_sorted["CQR"] , y_pred_sorted["CQR"], y_lower_sorted["CQR"], y_upper_sorted["CQR"]= sort_y_values(y_test, y_cqr_pred[stat_id], y_pis_cqr[stat_id])

    for idx, (axis, strat) in enumerate(zip(axs, strategy)):
        plot_prediction_intervals(
                strat,
                axis,
                y_test_sorted[strat],
                y_pred_sorted[strat],
                y_lower_sorted[strat],
                y_upper_sorted[strat],
                10,
                2,
                num_plots # num_plots
                )



lines_labels = [axis.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(_, []) for _ in zip(*lines_labels)]
plt.legend(
                      lines[:2] + lines[-2:], labels[:2] + labels[-2:],
                      loc='upper center',
                      bbox_to_anchor=(0, -0.15),
                      fancybox=True,
                      shadow=True,
                      ncol=2, fontsize = 20
                  )
for ax in ax.flat:
        ax.label_outer()
plt.show()